In [1]:
# Importar librerias
import pandas as pd

pd.set_option('display.float_format', '{:.0f}'.format)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import os

## Base

In [2]:
# Base Municipios
df_base = pd.read_excel("../Data/DIVIPOLA_Municipios.xlsx")

#
display(df_base.shape)
df_base.head()

(1122, 5)

,CodigoD,Departamento,CodigoM,Municipio,tipo
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio


In [3]:
df_base[df_base['CodigoM'] == 5001]

,CodigoD,Departamento,CodigoM,Municipio,tipo
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio


## Finanzas

In [4]:
# Fiananzas
df_finanzas = pd.read_csv("../Data/Fiananzascompilado.txt", sep="|", encoding="utf-8")

# 
display(df_finanzas.shape)
df_finanzas.head()

(1048574, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,76,Valle del Cauca,76000,Valle del Cauca,Finanzas públicas,Operaciones efectivas de caja,Ingresos totales,"18.325,30",NaN,2000,12,DNP a partir de información del FUT,Millones de pesos corrientes
1,76,Valle del Cauca,76001,Cali,Finanzas públicas,Operaciones efectivas de caja,Ingresos totales,"412.973,63",NaN,2000,12,DNP a partir de información del FUT,Millones de pesos corrientes
2,76,Valle del Cauca,76020,Alcalá,Finanzas públicas,Operaciones efectivas de caja,Ingresos totales,"3.951,56",NaN,2000,12,DNP a partir de información del FUT,Millones de pesos corrientes
3,76,Valle del Cauca,76036,Andalucía,Finanzas públicas,Operaciones efectivas de caja,Ingresos totales,"3.744,92",NaN,2000,12,DNP a partir de información del FUT,Millones de pesos corrientes
4,76,Valle del Cauca,76041,Ansermanuevo,Finanzas públicas,Operaciones efectivas de caja,Ingresos totales,"4.415,37",NaN,2000,12,DNP a partir de información del FUT,Millones de pesos corrientes


In [15]:
# Describe de los datos continuos
df_finanzas.describe()

,Código Departamento,Código Entidad,Dato Cualitativo,Año,Mes
count,1048574,1048574,0,1048574,1048574
mean,38,38113,NaN,2013,12
std,26,25999,NaN,7,2
min,1,1001,NaN,2000,0
25%,15,15667,NaN,2007,12
50%,25,25807,NaN,2014,12
75%,66,66000,NaN,2018,12
max,99,99773,NaN,2023,12


In [7]:
# dejar datos por municipio / son 23 años
df_finanzas[(df_finanzas['Año'] == 2000) ].pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index().head(5)

Indicador,Código Entidad,Entidad,% de ingresos corrientes destinados a funcionamiento,% de ingresos que corresponden a transferencias,Credito interno y externo,Déficit o ahorro corriente,Déficit o superávit total,Financiamiento,Funcionamiento,Gastos corrientes,Gastos corrientes per cápita,Gastos de capital (Inversión),Gastos de funcionamiento per cápita,Gastos totales,Gastos totales per cápita,Ingresos corrientes,Ingresos corrientes per cápita,Ingresos de capital,Ingresos no tributarios,Ingresos no tributarios per cápita,Ingresos per cápita por impuesto a la Industria y al comercio,Ingresos per cápita por impuesto predial,Ingresos totales,Ingresos totales per cápita,Ingresos tributarios,Ingresos tributarios per cápita,Intereses de deuda pública,Número de proyectos contratados,Número total de proyectos,Recursos del balance y variación de depósitos,Regalías per cápita (Valor efectivamente girado al municipio),Respaldo de la deuda,Transferencias de los ingresos corrientes,"Transferencias nacionales (SGP, etc.) per cápita",Transferencias per cápita de los ingresos corrientes
0,1001,Colombia,NaN,NaN,"-201,82","433,37","-1.092,35","1.092,35","2.670,67","3.062,26","74.879,32","7.354,20","70.355,80","10.416,46","289.530,14","3.495,63","67.131,21","5.807,30","530,97","13.985,12","6.187,83","10.238,40","9.324,10","295.574,20","2.606,85","25.198,23","391,59",NaN,NaN,"1.294,17","20.709,92",NaN,"348,30","177.434,20","27.947,87"
1,5000,Antioquia,"68,98","36,26","50,72","266,32","591,20","-591,20","4.332,93","4.903,08","98.902,88","6.263,94","91.852,20","11.167,02","324.510,84","5.169,41","95.201,04","6.588,81","1.078,79","26.555,63","12.189,18","15.280,86","11.758,22","329.800,76","3.687,85","38.243,65","570,15",NaN,NaN,"-641,91","7.937,84","29,74","402,76","179.732,28","30.401,76"
2,5001,Medellín,"68,81","18,95","3.129,87","5.748,63","45.430,88","-45.430,88","279.637,35","318.321,14","154.085,68","266.872,84","135.360,51","585.193,98","283.267,44","324.069,77","156.868,35","306.555,09","50.866,16","24.622,14","46.676,23","60.508,15","630.624,86","305.258,58","273.203,61","132.246,21","38.683,79",NaN,NaN,"-48.560,75","4,17","41,22","0,00","57.858,35","0,00"
3,5002,Abejorral,"126,68","71,72","-152,44","-408,22","-243,92","243,92","1.464,51","1.519,44","71.688,46","3.177,00","69.097,15","4.696,44","221.582,26","1.111,22","52.428,45","3.341,29","346,61","16.353,57","10.129,61","5.940,17","4.452,51","210.073,70","415,14","19.586,51","54,92",NaN,NaN,"396,36","1.264,17","2,49","349,47","134.172,12","16.488,37"
4,5004,Abriaquí,"170,24","70,99","542,15","-57,63","-362,44","362,44","350,48","353,12","118.855,27","1.487,62","117.966,34","1.840,74","619.569,17","295,49","99.458,09","1.182,82","87,76","29.538,20","2.031,30","11.620,67","1.478,31","497.578,26","41,67","14.025,24","2,64",NaN,NaN,"-179,71","2.547,63","1,94","166,06","297.357,79","55.894,65"


## Economia

In [12]:
# Economia
df_economia = pd.read_excel("../Data/Economia.xlsx")

# 
display(df_economia.shape)
df_economia.head()

(964622, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Economía,Valor agregado - Base 2015,"Agricultura, ganadería, caza, silvicultura y p...",NaN,NaN,2011,12,DNP a partir de información del DANE,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5001,Medellín,Economía,Valor agregado - Base 2015,"Agricultura, ganadería, caza, silvicultura y p...","0,07",NaN,2011,12,DNP a partir de información del DANE,Porcentaje (el valor está multiplicado por 100)
2,5,Antioquia,5002,Abejorral,Economía,Valor agregado - Base 2015,"Agricultura, ganadería, caza, silvicultura y p...","26,01",NaN,2011,12,DNP a partir de información del DANE,Porcentaje (el valor está multiplicado por 100)
3,5,Antioquia,5004,Abriaquí,Economía,Valor agregado - Base 2015,"Agricultura, ganadería, caza, silvicultura y p...","25,68",NaN,2011,12,DNP a partir de información del DANE,Porcentaje (el valor está multiplicado por 100)
4,5,Antioquia,5021,Alejandría,Economía,Valor agregado - Base 2015,"Agricultura, ganadería, caza, silvicultura y p...","68,98",NaN,2011,12,DNP a partir de información del DANE,Porcentaje (el valor está multiplicado por 100)


In [13]:
# Describe de los datos continuos
df_economia.describe()

,Código Departamento,Código Entidad,Año,Mes
count,964622,964622,964622,964622
mean,38,38586,2017,12
std,26,26334,4,0
min,1,1001,2000,12
25%,15,15686,2015,12
50%,25,25843,2018,12
75%,66,66318,2021,12
max,99,99773,2023,12


In [14]:
# dejar datos por municipio / son 23 años
df_economia[(df_economia['Año'] == 2000) ].pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index().head(5)

Indicador,Código Entidad,Entidad,"% Actividades de servicios sociales, comunales y personales","% Agricultura, ganadería, caza, silvicultura y pesca","% Comercio, reparación, restaurantes y hoteles",% Construcción,"% Establecimientos financieros, inmobiliarias y servicios a empresas",% Explotación de minas y canteras,% Impuestos,% Industria manufacturera,"% Suministro de electricidad, gas y agua","% Transporte, almacenamiento y comunicaciones",PIB,"PIB - Actividades de servicios sociales, comunales y personales","PIB - Agricultura, ganadería, caza, silvicultura y pesca","PIB - Comercio, reparación, restaurantes y hoteles",PIB - Construcción,"PIB - Establecimientos financieros, inmobiliarias y servicios a empresas",PIB - Explotación de minas y canteras,PIB - Impuestos,PIB - Industria manufacturera,"PIB - Suministro de electricidad, gas y agua","PIB - Transporte, almacenamiento y comunicaciones",PIB per cápita,PIB per cápita como porcentaje del promedio nacional,Porcentaje de contribución al PIB nacional
0,1001,Colombia,"17,38","8,31","12,91","4,18","20,63","5,93","7,19","13,93","3,27","6,27","208.531,00","36.252,00","17.320,00","26.927,00","8.710,00","43.024,00","12.361,00","15.002,00","29.054,00","6.816,00","13.065,00","5.175.036,27",NaN,"100,00"
1,5000,Antioquia,"15,22","7,92","13,44","3,86","22,37","1,51","7,79","17,82","4,40","5,66","28.758,00","4.378,00","2.277,00","3.866,00","1.110,00","6.434,00","433,00","2.240,00","5.126,00","1.265,00","1.629,00","5.436.385,33","122,48","13,79"
2,8000,Atlántico,"15,97","2,55","15,03","2,73","22,50","0,19","8,54","18,58","6,50","7,42","8.925,00","1.425,00","228,00","1.341,00","244,00","2.008,00","17,00","762,00","1.658,00","580,00","662,00","4.424.037,42","99,67","4,28"
3,13000,Bolívar,"15,90","7,42","13,11","3,40","12,86","1,67","14,62","21,39","3,65","5,98","7.061,00","1.123,00","524,00","926,00","240,00","908,00","118,00","1.032,00","1.510,00","258,00","422,00","3.938.896,62","88,74","3,39"
4,15000,Boyacá,"19,13","21,47","15,19","3,60","7,83","4,02","4,28","10,59","4,35","9,53","5.720,00","1.094,00","1.228,00","869,00","206,00","448,00","230,00","245,00","606,00","249,00","545,00","4.633.267,18","104,38","2,74"


## Educacion

In [17]:
# Economia
df_educacion = pd.read_excel("../Data/Educacion.xlsx")

# 
display(df_educacion.shape)
df_educacion.head()

(378680, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Educación,Acceso a la educación,Cobertura bruta en transición,"95,42",NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5000,Antioquia,Educación,Acceso a la educación,Cobertura bruta en transición,"78,62",NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
2,5,Antioquia,5001,Medellín,Educación,Acceso a la educación,Cobertura bruta en transición,"103,23",NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
3,5,Antioquia,5002,Abejorral,Educación,Acceso a la educación,Cobertura bruta en transición,"51,80",NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
4,5,Antioquia,5004,Abriaquí,Educación,Acceso a la educación,Cobertura bruta en transición,"96,00",NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


In [18]:
# Describe de los datos continuos
df_educacion.describe()

,Código Departamento,Código Entidad,Dato Cualitativo,Año,Mes
count,378680,378680,0,378680,378680
mean,38,38202,NaN,2014,12
std,26,26077,NaN,5,1
min,1,1001,NaN,2003,6
25%,15,15673,NaN,2010,12
50%,25,25815,NaN,2015,12
75%,66,66045,NaN,2019,12
max,99,99773,NaN,2022,12


In [ ]:
# dejar datos por municipio 
df_educacion[(df_educacion['Año'] == 2005) ].pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index().head(5)

Indicador,Código Entidad,Entidad,Cobertura bruta en educación - Total,Cobertura bruta en educación básica,Cobertura bruta en educación media,Cobertura bruta en educación primaria,Cobertura bruta en educación secundaria,Cobertura bruta en transición,Cobertura en educación superior,Cobertura neta en educación - Total,Cobertura neta en educación básica,Cobertura neta en educación media,Cobertura neta en educación primaria,Cobertura neta en educación secundaria,Cobertura neta en transición,Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once),Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)
0,1001,Colombia,"97,86","114,36","64,90","118,09","88,97","95,42","28,44","88,11","91,07","33,97","91,11","62,33","62,40","6,03","3,41"
1,5000,Antioquia,"98,21","112,47","68,60","118,98","92,27","78,62","29,42","88,63","91,44","36,20","94,28","63,45","65,15","6,41","3,27"
2,5001,Medellín,"113,50","119,56","87,08","127,69","113,84","103,23",NaN,"101,20","104,50","46,98","107,62","80,31","85,98",NaN,NaN
3,5002,Abejorral,"86,61","94,39","46,85","119,47","73,66","51,80",NaN,"78,21","83,46","20,59","96,32","48,06","35,05",NaN,NaN
4,5004,Abriaquí,"88,07","95,47","51,40","109,85","77,78","96,00",NaN,"77,55","83,21","21,50","88,26","46,30","78,00",NaN,NaN


## Inversion en Investicacion Ciencia

In [22]:
# Economia
df_InvCientifica = pd.read_excel("../Data/Inversion_investicacion_ciencia.xlsx")

# 
display(df_InvCientifica.shape)
df_InvCientifica.head()

(726, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,NaN,NaN,2000,12,MinCiencias,Porcentaje
1,5,Antioquia,5000,Antioquia,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,25",NaN,2000,12,MinCiencias,Porcentaje
2,8,Atlántico,8000,Atlántico,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,04",NaN,2000,12,MinCiencias,Porcentaje
3,13,Bolívar,13000,Bolívar,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,04",NaN,2000,12,MinCiencias,Porcentaje
4,15,Boyacá,15000,Boyacá,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,03",NaN,2000,12,MinCiencias,Porcentaje


In [23]:
# Describe de los datos continuos
df_InvCientifica.describe()

,Código Departamento,Código Entidad,Dato Cualitativo,Año,Mes
count,726,726,0,726,726
mean,52,51849,NaN,2010,12
std,31,31262,NaN,6,0
min,1,1001,NaN,2000,12
25%,20,20000,NaN,2005,12
50%,52,52000,NaN,2010,12
75%,81,81000,NaN,2016,12
max,99,99000,NaN,2021,12


In [26]:
# dejar datos por municipio / son 23 años
df_InvCientifica[(df_InvCientifica['Año'] == 2005) ].pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index().head(5)

Indicador,Código Entidad,Entidad,Inversión departamental en Investigación y Desarrollo (I+D) como porcentaje del Producto Interno Bruto (PIB)
0,1001,Colombia,NaN
1,5000,Antioquia,"0,27"
2,8000,Atlántico,"0,04"
3,13000,Bolívar,"0,05"
4,15000,Boyacá,"0,04"


## Mercado laboral

In [28]:
# Economia
df_mercadoLaboral = pd.read_excel("../Data/Mercado laboral.xlsx")

# 
display(df_mercadoLaboral.shape)
df_mercadoLaboral.head()

(201002, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Mercado laboral,Trabajadores cotizantes al sistema general de ...,Trabajadores cotizantes al sistema general de ...,"6.101.123,50",NaN,2009,12,FILCO- Ministerio del Trabajo,Trabajadores
1,5,Antioquia,5000,Antioquia,Mercado laboral,Trabajadores cotizantes al sistema general de ...,Trabajadores cotizantes al sistema general de ...,"1.055.785,50",NaN,2009,12,FILCO- Ministerio del Trabajo,Trabajadores
2,5,Antioquia,5001,Medellín,Mercado laboral,Trabajadores cotizantes al sistema general de ...,Trabajadores cotizantes al sistema general de ...,"764.862,56",NaN,2009,12,FILCO- Ministerio del Trabajo,Trabajadores
3,5,Antioquia,5002,Abejorral,Mercado laboral,Trabajadores cotizantes al sistema general de ...,Trabajadores cotizantes al sistema general de ...,"2.580,83",NaN,2009,12,FILCO- Ministerio del Trabajo,Trabajadores
4,5,Antioquia,5004,Abriaquí,Mercado laboral,Trabajadores cotizantes al sistema general de ...,Trabajadores cotizantes al sistema general de ...,"138,58",NaN,2009,12,FILCO- Ministerio del Trabajo,Trabajadores


In [29]:
# Describe de los datos continuos
df_mercadoLaboral.describe()

,Código Departamento,Código Entidad,Dato Cualitativo,Año,Mes
count,201002,201002,0,201002,201002
mean,37,37710,NaN,2013,12
std,26,25773,NaN,2,0
min,1,1001,NaN,2009,12
25%,15,15638,NaN,2011,12
50%,25,25781,NaN,2013,12
75%,63,63401,NaN,2015,12
max,99,99773,NaN,2016,12


In [31]:
# dejar datos por municipio / son 23 años
df_mercadoLaboral[(df_mercadoLaboral['Año'] == 2009) ].pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index().head(5)

Indicador,Código Entidad,Entidad,Participación de los trabajadores cotizantes - Hombres,Participación de los trabajadores cotizantes - Mujeres,Participación de los trabajadores cotizantes - Sin reporte de sexo,Participación de los trabajadores cotizantes con 17 años o menos entre el total,Participación de los trabajadores cotizantes entre 18 y 28 años entre el total,Participación de los trabajadores cotizantes entre 29 y 40 años entre el total,Participación de los trabajadores cotizantes entre 41 y 50 años entre el total,Participación de los trabajadores cotizantes entre 51 y 59 años entre el total,Participación de los trabajadores cotizantes entre 60 años o más entre el total,Participación de los trabajadores cotizantes sin reporte de edad entre el total,Trabajadores cotizantes - Hombres,Trabajadores cotizantes - Mujeres,Trabajadores cotizantes - Sin reporte de sexo,Trabajadores cotizantes al sistema general de seguridad social (promedio mensual),Trabajadores cotizantes con 17 años o menos,Trabajadores cotizantes entre 18 y 28 años,Trabajadores cotizantes entre 29 y 40 años,Trabajadores cotizantes entre 41 y 50 años,Trabajadores cotizantes entre 51 y 59 años,Trabajadores cotizantes entre 60 años o más,Trabajadores cotizantes sin reporte de edad
0,1001,Colombia,"56,33","43,66","0,01","0,12","26,59","35,28","23,18","10,82","4,01","0,00","3.436.840,50","2.663.927,50","355,50","6.101.123,50","7.129,33","1.622.296,88","2.152.731,00","1.414.303,88","660.188,25","244.459,75","14,42"
1,5000,Antioquia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1.055.785,50",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5001,Medellín,"56,77","43,23","0,00","0,09","28,37","33,82","23,69","10,71","3,33","0,00","434.185,00","330.648,50","29,08","764.862,56","687,50","216.974,75","258.659,92","181.207,92","81.897,09","25.435,08","1,00"
3,5002,Abejorral,"61,78","38,22","0,00","0,07","25,88","34,04","22,84","12,37","4,80","0,00","1.594,42","986,42","0,00","2.580,83","1,73","667,83","878,50","589,58","319,33","124,00","0,00"
4,5004,Abriaquí,"73,66","26,34","0,00","0,00","15,94","36,02","30,55","15,21","2,29","0,00","102,08","36,50","0,00","138,58","0,00","22,08","49,92","42,33","21,08","3,17","0,00"


## Final

In [32]:
# Espacio para concatenar los data set's cuando se definan los periodos de tiempo y campos